In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import librosa
import sys

from scipy.stats import skew
from scipy.stats import kurtosis, median_abs_deviation

sys.path.append('/Users/jaewone/developer/tensorflow/baby-cry-classification')

In [2]:
from constant.os import *
from utils.os import *
from utils.sound import *
from trans_data.get_state_list import get_state_file_list

In [3]:
def get_mfcc(file_path,
             sampling_freq=16000,
             mfcc_coef=40,
             mfcc_coef_retain=25,
             mfcc_window_duration=0.0232):
    """
    오디오 파일에 대한 MFCC를 분석한 뒤 값을 반환한다.

    Parameters:
        file_path:            오디오 파일의 경로
        sampling_freq:        sampling rate
        mfcc_coef:            frame의 길이를 결정하는 파라미터.
        mfcc_coef_retain:     유지되는 MFCC 값의 길이
        mfcc_window_duration: 데이터를 읽을 때 겹쳐 읽는 길이(sec)

    Returns:
        list[0]: 초반 mfcc_coef_retain 만큼의 MFCC 값
        list[1]: 평균, 분산, 표준편차와 같은 MFCC의 특성값
    """

    # load wav file and normalize
    wave, sr = librosa.load(file_path, mono=True, sr=sampling_freq)
    wave = librosa.util.normalize(wave)

    # feature extraction
    """
    ① sr: sampling rate
    ② n_mfcc=20 : mfcc의 개수
    ③ n_fft=25
        frame의 길이를 결정하는 파라미터 입니다. n_fft를 설정하면 window size가 자동으로 같은 값으로 설정되는데 window size의 크기로 잘린 음성이 n_fft보다 작은 경우 0으로 padding을 붙여주는 작업을 하기 때문에 n_fft는 window size보다 크거나 같아야 한다.
        보통 n_fft = sr * frame_length 로 설정
    ④ hop_length=10
        hop_length의 길이만큼 옆으로 가면서 데이터를 읽는다. 
        10ms를 기본으로 하고 있어 16000Hz인 음성에서는 160에 해당한다. (16000 * 0.01 = 160)
        보통 hop_length는 = sr * frame_stride 로 설정한다.
    """
    mfccs = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=mfcc_coef, hop_length=int(
        mfcc_window_duration*sr/2.0), n_fft=int(mfcc_window_duration*sr))

    # 정규화: 평균이 0, 표준편차 1
    mfccs = (mfccs - np.mean(mfccs))/np.std(mfccs)

    # keep the first mfcc_coef_retain coefficients
    """
    일부 계수를 폐기하는 이유는 무엇입니까? 
    일반적으로 하위 MFCC는 스펙트럼의 전체 모양에 대한 더 많은 정보를 포함하고(따라서 더 중요함) 상위 MFCC는 스펙트럼에서 더 미세한 세부 사항을 나타내기 시작합니다. 
    처음 25개의 MFCC만 유지하도록 선택함으로써 작성자는 마지막 15개의 계수(26에서 40까지)가 모델의 증가된 복잡성을 정당화할 만큼 중요한 정보를 충분히 전달하지 않는다고 결정했을 가능성이 큽니다.
    즉, 유지할 MFCC 수의 선택은 문제에 따라 다를 수 있으며 다른 응용 프로그램이나 모델에는 다른 수가 필요할 수 있습니다. 일부 응용 프로그램의 경우 처음 13개의 계수만 사용되는 반면 
    다른 응용 프로그램의 경우 40개 모두 사용될 수 있습니다. 주어진 작업에 대한 최적의 수를 찾으려면 종종 어느 정도의 실험이 필요합니다.
    """
    mfccs = mfccs[:mfcc_coef_retain, :]

    # calculate MFCC statistics
    mfccs_min = mfccs.min(axis=1)
    mfccs_max = mfccs.max(axis=1)
    mfccs_median = np.median(mfccs, axis=1)
    mfccs_mean = np.mean(mfccs, axis=1)
    mfccs_var = np.var(mfccs, axis=1)
    mfccs_skewness = skew(mfccs, axis=1)
    mfccs_kurtosis = kurtosis(mfccs, axis=1)
    mfccs_mad = median_abs_deviation(mfccs, axis=1)

    mfccs_first_derivative = np.diff(mfccs, n=1, axis=1)
    mfccs_first_derivative_mean = np.mean(mfccs_first_derivative, axis=1)
    mfccs_first_derivative_var = np.var(mfccs_first_derivative, axis=1)

    mfccs_second_derivative = np.diff(mfccs, n=2, axis=1)
    mfccs_second_derivative_mean = np.mean(mfccs_second_derivative, axis=1)
    mfccs_second_derivative_var = np.var(mfccs_second_derivative, axis=1)

    mfccs_stats = np.vstack((mfccs_min, mfccs_max, mfccs_median, mfccs_mean, mfccs_var, mfccs_skewness, mfccs_kurtosis, mfccs_mad,
                            mfccs_first_derivative_mean, mfccs_first_derivative_var, mfccs_second_derivative_mean, mfccs_second_derivative_var))

    return pd.Series([mfccs, mfccs_stats.transpose()])

In [4]:
file_list = get_state_file_list(data_path)

df = pd.DataFrame({'file':file_list})
df['duration'] = df['file'].apply(lambda file: get_duration(file))
df['file'] = df['file'].apply(lambda file: file.rsplit('/', 1)[1])
df['state'] = df['file'].apply(lambda file: file.split('_', 1)[0]).astype('category')
df.tail(3)

/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


,file,duration,state
3777,uncomfortable_109.wav,5.0,uncomfortable
3778,uncomfortable_135.wav,5.0,uncomfortable
3779,uncomfortable_121.wav,5.0,uncomfortable


In [5]:
# Get MFCC data
from tqdm import tqdm

tqdm.pandas()
df[['mfccs', 'mfccs_stats']] = df.progress_apply(lambda x: get_mfcc(os.path.join(data_path, x['state'], x['file'])), axis=1)
df.tail(3)

  0%|          | 0/3780 [00:00<?, ?it/s]

100%|██████████| 3780/3780 [00:29<00:00, 126.41it/s]


,file,duration,state,mfccs,mfccs_stats
3777,uncomfortable_109.wav,5.0,uncomfortable,"[[-5.220834, -6.01142, -6.1456327, -5.8742223,...","[[-8.55258, -3.0695317, -5.640008, -5.645126, ..."
3778,uncomfortable_135.wav,5.0,uncomfortable,"[[-3.8128958, -3.598897, -3.5140378, -3.429702...","[[-11.446929, -2.9981005, -4.655477, -5.477557..."
3779,uncomfortable_121.wav,5.0,uncomfortable,"[[-6.8476195, -6.789292, -6.7553797, -6.767772...","[[-7.3525023, -2.7414045, -5.8861413, -5.91529..."


In [6]:
# state 열을 카테고리 타입으로 변환한 다음 int 형태로 캐스팅한다.
df.state = df.state.astype('category')
df = df.assign(state_code=df.state.cat.codes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   file         3780 non-null   object  
 1   duration     3780 non-null   float64 
 2   state        3780 non-null   category
 3   mfccs        3780 non-null   object  
 4   mfccs_stats  3780 non-null   object  
 5   state_code   3780 non-null   int8    
dtypes: category(1), float64(1), int8(1), object(3)
memory usage: 126.0+ KB


In [7]:
def split_xy(train_df: pd.DataFrame, test_df:pd.DataFrame):
    x_train = np.array(train_df['mfccs_stats'].to_list())
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2])

    y_train = np.array(train_df['state_code'].to_list())
    y_train = y_train.reshape(y_train.shape[0], 1)

    x_test = np.array(test_df['mfccs_stats'].to_list())
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2])

    y_test = np.array(test_df['state_code'].to_list())
    y_test = y_test.reshape(y_test.shape[0], 1)

    return x_train, x_test, y_train, y_test

In [8]:
def get_lstm_model(num_classes):

    # model = tf.keras.Sequential([tf.keras.layers.LSTM(256, return_sequences=False),
    #                           tf.keras.layers.BatchNormalization(),
    #                           tf.keras.layers.Dropout(0.4),
    #                           tf.keras.layers.Dense(num_classes, activation='softmax')])

    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(256, return_sequences=True),  # 첫 번째 LSTM 레이어
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.LSTM(128, return_sequences=False),  # 두 번째 LSTM 레이어
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])


    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=[
                  'sparse_categorical_accuracy'])

    return model

In [9]:
EPOCHS = 10000

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

X_train, X_test, Y_train, Y_test = split_xy(train, test)

lstm_model = get_lstm_model(num_classes=10)

history = lstm_model.fit(X_train, Y_train, batch_size=128, epochs=EPOCHS, validation_data=(
    X_test, Y_test), callbacks=[], verbose=0)

accuracy_score = lstm_model.evaluate(X_test, Y_test)

2023-08-08 16:26:32.471588: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-08 16:26:32.472100: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-08-08 16:26:32.595244: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-08 16:26:34.288577: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-08-08 16:26:34.689584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-08-08 16:26:34.858867: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-08-08 16:26:35.085858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-08-08 16:26:35.348894: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-08-08 16:26:36.955340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report
lstm_test_preds = lstm_model.predict(X_test)
lstm_test_pred_classes = np.argmax(lstm_test_preds, axis=1)

print(classification_report(Y_test, lstm_test_pred_classes))